<a href="https://colab.research.google.com/github/wiatrak2/BScThesis/blob/master/svhnConceptors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append('BScThesis/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!pip3 install torch torchvision

    100% |████████████████████████████████| 519.5MB 30kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x59dbe000 @  0x7f26747ae2a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641
    100% |████████████████████████████████| 61kB 21.4MB/s 
    100% |████████████████████████████████| 2.0MB 4.7MB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0


In [3]:
!git clone https://github.com/wiatrak2/BScThesis

Cloning into 'BScThesis'...
remote: Enumerating objects: 104, done.
remote: Counting objects: 100% (104/104), done.
remote: Compressing objects: 100% (86/86), done.
remote: Total 283 (delta 56), reused 47 (delta 18), pack-reused 179
Receiving objects: 100% (283/283), 39.02 MiB | 5.88 MiB/s, done.
Resolving deltas: 100% (155/155), done.


In [0]:
from importlib import reload 

In [0]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [0]:
import torchvision
import torchvision.transforms as transforms

In [0]:
import matplotlib.pyplot as plt
import pickle
import sympy
from collections import defaultdict, namedtuple
import copy

In [0]:
from mnist_m import Mnist_M
import datasets_merge
import models
import svhn_models
import trainer
import domain_trainer
from test_model import test_model
import plots
import test
import plotly_plots

In [0]:
def convertMnist(img):
  img = img.squeeze().numpy()
  return torch.from_numpy(np.tile(img, (3,1,1)))

In [0]:
batch_size=128
num_workers=1

In [12]:
mnist_transform=transforms.Compose([
                           transforms.Resize((32,32)),    
                           transforms.ToTensor(),
                           transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           transforms.Lambda(convertMnist)
                       ])

mnist_train_set = torchvision.datasets.MNIST(root='./data', train=True, download=True,
                                     transform=mnist_transform)
mnist_train_loader = torch.utils.data.DataLoader(mnist_train_set, batch_size=batch_size,
                                         shuffle=True, num_workers=num_workers)

mnist_test_set = torchvision.datasets.MNIST(root='./data', train=False, download=True,
                                     transform=mnist_transform)
mnist_test_loader = torch.utils.data.DataLoader(mnist_test_set, batch_size=batch_size,
                                         shuffle=False, num_workers=num_workers)




Processing...
Done!


In [13]:
svhn_transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                       ])

svhn_train_set = torchvision.datasets.SVHN(root='./data', download=True,
                                     transform=svhn_transform)
svhn_train_loader = torch.utils.data.DataLoader(svhn_train_set, batch_size=batch_size,
                                         shuffle=True, num_workers=num_workers)

svhn_test_set = torchvision.datasets.SVHN(root='./data',  download=True,
                                     transform=svhn_transform)
svhn_test_loader = torch.utils.data.DataLoader(svhn_test_set, batch_size=batch_size,
                                         shuffle=False, num_workers=num_workers)





Using downloaded and verified file: ./data/train_32x32.mat


In [0]:
merge_train_set = datasets_merge.Merge_Datasets([mnist_train_set, svhn_train_set], get_labels=False)
merge_test_set = datasets_merge.Merge_Datasets([mnist_test_set, svhn_test_set], get_labels=False)
merge_train_loader = torch.utils.data.DataLoader(merge_train_set, 
                                                   batch_size=batch_size,
                                         shuffle=True, num_workers=num_workers)
merge_test_loader = torch.utils.data.DataLoader(merge_test_set, 
                                                   batch_size=batch_size,
                                         shuffle=True, num_workers=num_workers)

In [0]:
log_interval = 100

In [0]:
use_cuda = True
use_cuda = use_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [0]:
lr = 0.01
momentum=0.9

In [0]:
Models = namedtuple('Models', ['model_f', 'model_c', 'model_d'])
Optims = namedtuple('Optims', ['optim_f', 'optim_c', 'optim_d'])
Criterions = namedtuple('Criterions', ['criterion', 'criterion_domain'])
Loaders = namedtuple('Loaders', ['train_loader', 'train_loader_domain',
                                'source_test_loader', 'target_test_loader',
                                'merged_test_loader'])

In [0]:
loaders = Loaders(svhn_train_loader, mnist_train_loader, 
                 svhn_test_loader, mnist_test_loader, merge_test_loader)

In [0]:
criterion = F.nll_loss
criterion_domain = F.nll_loss

In [0]:
criterions = Criterions(criterion, criterion_domain)

In [0]:
epochs=15

In [0]:
model_f = svhn_models.SvhnFeatureExtractor([3,48, 64, 128, 160, 192], pool_stride=2, pool_padding=1).to(device)

In [24]:
output_size = model_f.get_output_size(device)[1]
output_size

768

In [0]:
model_f_3 = nn.Sequential(nn.Linear(output_size, 320), nn.LeakyReLU(), nn.Linear(320,3))
model_f = models.extend_feature_extractor(model_f, 
                                         model_f_3, False, False).to(device)

In [0]:
model_c = models.MnistClassPredictor(input_size=3).to(device)
model_d = models.MnistDomain(input_size=3).to(device)

In [0]:
optim_f = optim.SGD(model_f.parameters(), lr=lr, momentum=momentum, nesterov=True)
optim_c = optim.SGD(model_c.parameters(), lr=lr, momentum=momentum, nesterov=True)
optim_d = optim.SGD(model_d.parameters(), lr=lr, momentum=momentum, nesterov=True)

In [0]:
train_models = Models(model_f, model_c, model_d)
optims = Optims(optim_f, optim_c, optim_d)

In [0]:
model_trainer = trainer.Trainer(train_models, optims, criterions, device,
                                train_domain=False, tune_lr=True)

In [35]:
test_history = defaultdict(lambda:[])
model_trainer.train(epochs, loaders, test_history=test_history)

Train Epoch: 					1 [0/73257 (0%)]	Loss: 2.400777, lr: 0.01000 lambd: 1.00000
Train Epoch: 					1 [12800/73257 (17%)]	Loss: 2.261837, lr: 0.00921 lambd: 1.00000
Train Epoch: 					1 [25600/73257 (35%)]	Loss: 2.218988, lr: 0.00855 lambd: 1.00000
Train Epoch: 					1 [38400/73257 (52%)]	Loss: 2.259705, lr: 0.00799 lambd: 1.00000
Train Epoch: 					1 [51200/73257 (70%)]	Loss: 2.161829, lr: 0.00751 lambd: 1.00000
Train Epoch: 					1 [64000/73257 (87%)]	Loss: 2.155825, lr: 0.00709 lambd: 1.00000

Target Domain Test set: Average loss: 2.3539, Accuracy: 1726/10000 (17%)
Source Domain Test set: Average loss: 1.9672, Accuracy: 21334/73257 (29%)
Domains predictor:  Accuracy: 31357/83257 (38%)

Train Epoch: 					2 [0/73257 (0%)]	Loss: 2.059867, lr: 0.00682 lambd: 1.00000
Train Epoch: 					2 [12800/73257 (17%)]	Loss: 1.878447, lr: 0.00648 lambd: 1.00000
Train Epoch: 					2 [25600/73257 (35%)]	Loss: 1.808352, lr: 0.00618 lambd: 1.00000
Train Epoch: 					2 [38400/73257 (52%)]	Loss: 1.794821, lr: 0.0059

In [0]:
best_mf = copy.deepcopy(model_f)

In [0]:
def get_input_mtx(loader, model, c_size):
  X = np.zeros(c_size).T
  for batch_idx, (data, labels) in enumerate(loader):
    data = data.to(device)
    idx = batch_idx * loader.batch_size
    X[idx:idx+loader.batch_size] = model(data).cpu().detach().numpy()
  return X.T

In [0]:
def get_conceptor(X, alpha):
  R = np.matmul(X, X.T) * (1/X.shape[0])
  IdX = np.identity(X.shape[0])
  return np.matmul(R, np.linalg.inv((R + alpha**(-2) * IdX)))

In [0]:
def quota(C):
  U, S, U_T = np.linalg.svd(C)
  return np.sum(S) / len(S)

In [0]:
def get_by_label(dataset, label_set, label):
  res = []
  for i, data in enumerate(label_set):
    if data[1] == label:
      res.append(dataset[i])
  return res

In [0]:
X = get_input_mtx(svhn_test_loader, best_mf, (3, len(svhn_test_set)))

In [0]:
X_m = get_input_mtx(mnist_test_loader, best_mf, (3, len(mnist_test_set)))

In [0]:
X0 = X[:,:10000]

In [0]:
C = get_conceptor(X0, 0.01)

In [0]:
C_m = get_conceptor(X_m, 0.01)

In [56]:
quota(C)

0.8433390074961128

In [57]:
quota(C_m)

0.7699748243961263

In [0]:
X_0 = get_by_label(X.T, svhn_test_set, 0)
X_0 = np.array(X_0).T

In [67]:
X_0.shape

(3, 4948)

In [0]:
X_m_0 = get_by_label(X_m.T, mnist_test_set, 0)
X_m_0 = np.array(X_m_0).T

In [0]:
import plotly
from plotly import tools
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go

In [68]:
[x,y,z] = X0
trace1 = go.Scatter3d(x=x,y=y,z=z, mode='markers', marker=dict(
        size=12,line=dict(color='rgba(217, 217, 217, 0.14)',width=0.5),opacity=0.8))

[x2, y2, z2] = X_m
trace2 = go.Scatter3d(x=x2,y=y2,z=z2,mode='markers',marker=dict(
        color='rgb(127, 127, 127)',size=12,symbol='circle',line=dict(color='rgb(204, 204, 204)',width=1),opacity=0.9))

[x3, y3, z3] = X_0
trace3 = go.Scatter3d(x=x3,y=y3,z=z3,mode='markers',marker=dict(color='rgb(255, 255, 204)',
        size=12,symbol='circle',line=dict(color='rgb(255, 255, 20)',width=1),opacity=0.9))

[x4, y4, z4] = X_m_0
trace4 = go.Scatter3d(x=x4,y=y4,z=z4,mode='markers',marker=dict(color='rgb(0, 255, 204)',
        size=12,symbol='circle',line=dict(color='rgb(0, 255, 20)',width=1),opacity=0.9))

data = [trace1, trace2, trace3, trace4]
layout = go.Layout(margin=dict(l=0,r=0,b=0,t=0))
fig = go.Figure(data=data, layout=layout)

plotly_plots.configure_plotly_browser_state()

init_notebook_mode(connected=False)

fig['layout'].update(height=800, width=1200, title='MNIST')
plotly.offline.iplot(fig, filename='basic-line')